In [31]:
#Code to generate the required digester size
import pandas as pd
import numpy as np
result = pd.read_csv("../analysis/Household_Survey_Analysis_with_derivations.csv") 
print(result.head())
# Constants (from literature)
Y = 0.3  # m³/kg biogas yield per kg of VS, 
VS = 0.8  # Volatile Solids content (fraction of total solids)
retention_time_days = 30  # days

  House ID  Household Size Primary Fuel  Fuel mass (KG)  Time taken (days)  \
0     W001               3          LPG            12.5                 45   
1     W002               4          LPG            12.5                 40   
2     W003               4          LPG            12.5                 42   
3     W004               1          LPG             3.0                 50   
4     W005               5          LPG            12.5                 30   

   Solid waste/day (Kg)  Daily Fuel Usage (KG/day)  \
0                   1.2                   0.277778   
1                   1.8                   0.312500   
2                   0.8                   0.297619   
3                   0.3                   0.060000   
4                   2.7                   0.416667   

   Gross Energy per day (MJ/day)  Useful Energy per day (MJ/day)  \
0                      12.805556                        7.043056   
1                      14.406250                        7.923438   
2 

In [32]:
kitchen_waste = result['Solid waste/day (Kg)'].sum()
print(f"Total Solid Waste: {kitchen_waste:.2f} kg/day")

def total_biogas_per_day (kitchen_waste, VS, Y):
    return kitchen_waste * VS * Y

kitchen_biogas_per_day = total_biogas_per_day(kitchen_waste, VS, Y)

#Total biogas yield from sum of daily kitchen waste in Wumba
print(f"\nTotal biogas production potential from kitchen waste: {kitchen_biogas_per_day:.2f} m^3/day")

Total Solid Waste: 36.50 kg/day

Total biogas production potential from kitchen waste: 8.76 m^3/day


In [ ]:
total_useful_energy = result['Useful Energy per day (MJ/day)'].sum()

print(f"Total Useful Energy Demand: {total_useful_energy:.2f} MJ/day")

# Factors for digester sizing
biogas_energy_content = 20 # MJ/m^3 biogas (as specified by user)
biogas_stove_efficiency = 0.55 # from literature 
digester_hydraulic_retention_time = 30 # from literature (days)
design_buffer = 1.5 # community safety factor 

def required_biogas_yield(total_useful_energy, biogas_energy_content, biogas_stove_efficiency):
    return total_useful_energy / (biogas_energy_content * biogas_stove_efficiency)

Wumba_biogas_need = required_biogas_yield(total_useful_energy, biogas_energy_content, biogas_stove_efficiency)
print(f"\nRequired daily biogas to meet energy demand: {Wumba_biogas_need:.2f} m^3/day")

# --- Incorporating other waste sources ---
#Estimate the daily amount and biogas potential of other waste sources:
#
estimated_daily_cow_dung = 50 #kg from estimates of cattle in the area 
biogas_yield_cow_dung = 0.15 # m^3 biogas per kg of cow dung (literature)
cow_dung_biogas = estimated_daily_cow_dung * biogas_yield_cow_dung

estimated_daily_meat_waste = 30 #kg from abattoir estimates
biogas_yield_meat_waste = 0.3 # m^3 biogas per kg of meat waste (literature)
meat_waste_biogas = estimated_daily_meat_waste * biogas_yield_meat_waste

total_waste_biogas_potential = kitchen_biogas_per_day + cow_dung_biogas + meat_waste_biogas
print(f"Total daily biogas potential from all waste sources: {total_waste_biogas_potential:.2f} m^3/day")

#Total Solids (TS%) in the Digester
#Assuming kitchen waste is 20% solids, cow dung is 15% solids, and meat waste is 30% solids
total_solids = (kitchen_waste * 0.2) + (estimated_daily_cow_dung * 0.15) + (estimated_daily_meat_waste * 0.3)
print(f"Total solids in the digester: {total_solids:.2f} kg/day")

#Reactor Slurry Target 
reactor_slurry_target = 0.08 # %TS, typical for biogas digesters
slurry_volume_needed = total_solids / (reactor_slurry_target * 1000 ) #(in m^3/day)
print(f"Reactor slurry volume: {slurry_volume_needed:.2f} m^3/day")

# Calculate the required digester volume based on Hydraulic Retention Time (HRT)
# Required Digester Volume = Total Wet Waste Input per Day * Hydraulic Retention Time * Design Buffer
# Assuming a rough estimate of total wet waste input (solid waste + other sources) and converting kg to m^3 with an assumed density (adjust as needed)
estimated_total_wet_waste = kitchen_waste + estimated_daily_cow_dung + estimated_daily_meat_waste # include other sources here
assumed_wet_waste_density = 1000 #( kg/m^3-density of water, adjust for waste)
estimated_total_wet_waste_input_m3_per_day = estimated_total_wet_waste / assumed_wet_waste_density

required_digester_volume = slurry_volume_needed * digester_hydraulic_retention_time * design_buffer

print(f"Biogas digester volume: {required_digester_volume:.2f} m^3")

# Compare the total biogas potential with the required yield to see if it's sufficient.
print(f"\nIs total waste biogas potential sufficient?")
if total_waste_biogas_potential >= Wumba_biogas_need:
    print("The biogas potential from available waste meets the community's energy needs.")
else:
    print("The biogas potential from available waste does NOT meet the community's energy needs. \n Consider additional waste sources or system optimizations.")

Total Useful Energy Demand: 233.61 MJ/day

Required daily biogas to meet energy demand: 21.24 m^3/day
Total daily biogas potential from all waste sources: 25.26 m^3/day
Total solids in the digester: 23.80 kg/day
Reactor slurry volume: 0.30 m^3/day
Biogas digester volume: 13.39 m^3

Is total waste biogas potential sufficient?
The biogas potential from available waste meets the community's energy needs.
